## Issue Credential

Issuing a credential involves establishing a connection between the issuer and the holder, which is done by following the process outlined in *Example 01 - Connections*. Once the connection is established, the issuer will prepare and send a credential offer, which creates an issue record on both the issuer's and holder's agents. The holder will then retrieve the list of issue records, find the one they wish to accept, and notify the issuer of their acceptance. Finally, the issuer will issue the credential to the holder, completing the process.

In [22]:
import os
import time
import datetime
import base64
from pprint import pprint
from dotenv import load_dotenv
from termcolor import colored,cprint

from prism_agent_open_api_specification_client import Client
from prism_agent_open_api_specification_client.types import Response
from prism_agent_open_api_specification_client.models import ConnectionCollection,Connection,ConnectionInvitation,CreateConnectionRequest,AcceptConnectionInvitationRequest
from prism_agent_open_api_specification_client.api.connections_management import get_connections,get_connection,create_connection,accept_connection_invitation
from prism_agent_open_api_specification_client.models import IssueCredentialRecord, CreateIssueCredentialRecordRequest, CreateIssueCredentialRecordRequestClaims, IssueCredentialRecordCollection, IssueCredentialRecordAllOfProtocolState
from prism_agent_open_api_specification_client.api.issue_credentials_protocol import get_credential_record, get_credential_records, create_credential_offer,accept_credential_offer,issue_credential
from prism_agent_open_api_specification_client.models import CreateManagedDidRequestDocumentTemplatePublicKeysInnerPurpose, CreateManagedDidRequestDocumentTemplatePublicKeysInner
from prism_agent_open_api_specification_client.models import CreateManagedDidRequestDocumentTemplate, CreateManagedDidRequest, CreateManagedDIDResponse
from prism_agent_open_api_specification_client.models import ListManagedDIDResponseInner, ListManagedDIDResponseInnerStatus
from prism_agent_open_api_specification_client.models import DIDDocumentMetadata, DIDOperationResponse, DidOperationSubmission, DIDResponse, DID, Service, ServiceType   
from prism_agent_open_api_specification_client.api.did_registrar import create_managed_did, list_managed_did, publish_managed_did  


### Ultilitary functions

In [11]:
def get_invitation_str(connection):
    parts = connection.invitation.invitation_url.split("=")
    return parts[1]

def find_credential_record_by_state(client, state):
    holder_credential_records: Response[IssueCredentialRecordCollection] = get_credential_records.sync(client=client)

    for offer in holder_credential_records.items:
        if(offer.protocol_state == state):
            return offer
    return None 

def print_credential_record(credential_record):
    print(f"record_id:          {credential_record.record_id}")
    print(f"subject_id:         {credential_record.subject_id}")
    print(f"role:               {credential_record.role}")
    print(f"protocol_state:     {credential_record.protocol_state}")
    print(f"publication_state:  {credential_record.publication_state}")
    print(f"created_at:         {credential_record.created_at}")
    print(f"updated_at:         {credential_record.updated_at}")
    
    
def print_connection(connection):
    print(f"connection_id: {connection.connection_id}")
    print(f"state:         {connection.state}")
    print(f"label:         {connection.label}")
    print(f"my_did:        {connection.my_did}")
    print(f"their_did:     {connection.their_did}")
    print(f"created_at:    {connection.created_at}")

### Client instances

We will create two separate clients, one for the issuer and one for the holder, in order to establish a connection between the two.

note: remember to update the file variables.env with the URLs and API keys provided to you.


In [3]:
load_dotenv("../BetaProgram/variables.env")
issuerApiKey = os.getenv('ISSUER_APIKEY')
issuerUrl = os.getenv('ISSUER_URL')

holderApiKey = os.getenv('HOLDER_APIKEY')
holderUrl = os.getenv('HOLDER_URL')

issuer_client = Client(base_url=issuerUrl, headers={"apiKey": issuerApiKey})
holder_client = Client(base_url=holderUrl, headers={"apiKey": holderApiKey})

### Create connection

For details on this see "Example 01 - Connections"

In [4]:
print("Please wait...")

conn_request = CreateConnectionRequest()
conn_request.label = f'Issue credential {datetime.date.today().strftime("%Y-%m-%d %H:%M:%S")}'
issuer_connection: Response[Connection] =  create_connection.sync(client=issuer_client,json_body=conn_request)

invitation = get_invitation_str(issuer_connection)

accept_conn_request = AcceptConnectionInvitationRequest(invitation)
holder_connection: Response[ConnectionInvitation] =  accept_connection_invitation.sync(client=holder_client,json_body=accept_conn_request)


issuer_connection: Response[Connection] = get_connection.sync(client=issuer_client,connection_id=issuer_connection.connection_id)
holder_connection: Response[Connection] = get_connection.sync(client=holder_client,connection_id=holder_connection.connection_id)

while (issuer_connection.state != 'ConnectionResponseSent' and holder_connection.state != 'ConnectionResponseReceived'):
    issuer_connection: Response[Connection] = get_connection.sync(client=issuer_client,connection_id=issuer_connection.connection_id)
    holder_connection: Response[Connection] = get_connection.sync(client=holder_client,connection_id=holder_connection.connection_id)
    print("Issuer State: {} / Holder State: {} \n".format(issuer_connection.state,holder_connection.state))
    time.sleep(1)
    
print("Connection established between Issuer and Holder!")
print("\nIssuer connection:\n")
print_connection(issuer_connection)
print("\nHolder connection:\n")
print_connection(holder_connection)

Please wait...
Issuer State: InvitationGenerated / Holder State: ConnectionRequestPending 

Issuer State: ConnectionResponsePending / Holder State: ConnectionRequestPending 

Issuer State: ConnectionResponsePending / Holder State: ConnectionRequestSent 

Issuer State: ConnectionResponsePending / Holder State: ConnectionRequestSent 

Issuer State: ConnectionResponsePending / Holder State: ConnectionResponseReceived 

Connection established between Issuer and Holder!

Issuer connection:

connection_id: 1fdfca4d-bf32-46e7-af5c-c2115ecc22ec
state:         ConnectionResponsePending
label:         Issue credential 2023-01-29 00:00:00
my_did:        did:peer:2.Ez6LSqN1jSpye52Fs3xUz8L4KqPvWXLgGpQpgUCws3V4xtYxK.Vz6Mkrc6eD4pUWwEGB4Nq7C6uEwUSCnX5NivvbWAn1wZQxLHr.SeyJ0IjoiZG0iLCJzIjoiaHR0cHM6Ly9iZzY1ai5hdGFsYXByaXNtLmlvL3ByaXNtLWFnZW50L2RpZGNvbW0iLCJyIjpbXSwiYSI6WyJkaWRjb21tL3YyIl19
their_did:     did:peer:2.Ez6LSpzdmF4wrt3MEuMPxWTdL9TRkpTmKdgq3zqPsgEZ4giLi.Vz6MkgXLUEFc3LHcQ8kfA5c3DYhtXCkWBTSBuhF2

### Holder - Create PRISM DID (TBD)


In [5]:
public_key = CreateManagedDidRequestDocumentTemplatePublicKeysInner(id="key1", purpose=CreateManagedDidRequestDocumentTemplatePublicKeysInnerPurpose.AUTHENTICATION)
service = Service(id="service1", type=ServiceType.MEDIATORSERVICE, service_endpoint=[f"{holderUrl}/didcomm"])
document_template = CreateManagedDidRequestDocumentTemplate([public_key],[])
did_request = CreateManagedDidRequest(document_template)

did_subject: Response[CreateManagedDIDResponse] = create_managed_did.sync(client=holder_client, json_body=did_request)

pprint(f"{did_subject.long_form_did}")

'did:prism:e62b4529af0964e37983af7abc05f5ddfefa6b317a4f685e8cecdc950ad079b7:CrwBCrkBElkKBGtleTEQBEJPCglzZWNwMjU2azESIJijkDX4d_iqN8wBfrh5Ne4q6qlSOc59HrjQoLw_TL3oGiA0OmQ6rl1M5FBzl6IvKeyX0rZZR-KWg-4eTzxXiOIExhJcCgdtYXN0ZXIwEAFCTwoJc2VjcDI1NmsxEiA-0cbEBtYruG-pEqvXtgfePaafWmMAXGKcI_armAGlAxogyay3a2aFUDckUl1alQQNJzEMF4mEw8eflR-cY1zpklo'


### Issuer - Create credential claim

The issuer creates the credential claim object to store the attributes that will be part of the verifiable credential.

In [6]:
data = {
        "firstname": 'Joe',
        "lastname": 'Soap',
        "birthdate": '11/11/1999'
      }

credential_claims = CreateIssueCredentialRecordRequestClaims().from_dict(data)

### Issuer - Send credential offer

The issuer creates a `CreateIssueCredentialRecordRequest`. It contains the `subject_id`, the `claims` and other metadata. This object is passed to the `create_credential_offer` endpoint.
The `create_credential_offer` call creates an `IssueCredentialRecord` on the issuer side, it also sends the credential offer to the holder.

In [7]:
credential_offer = CreateIssueCredentialRecordRequest(subject_id=issuer_connection.their_did, claims=credential_claims, schema_id="1234", validity_period=3600, automatic_issuance=False, await_confirmation=False)

issuer_credential_offer: Response[IssueCredentialRecord] = create_credential_offer.sync(client=issuer_client,json_body=credential_offer)

print_credential_record(issuer_credential_offer)

record_id:          6d02900b-d861-4d2e-a516-d4486424e0fb
subject_id:         did:peer:2.Ez6LSpzdmF4wrt3MEuMPxWTdL9TRkpTmKdgq3zqPsgEZ4giLi.Vz6MkgXLUEFc3LHcQ8kfA5c3DYhtXCkWBTSBuhF2m2Qinv8Jm.SeyJ0IjoiZG0iLCJzIjoiaHR0cHM6Ly9sdzIycS5hdGFsYXByaXNtLmlvL3ByaXNtLWFnZW50L2RpZGNvbW0iLCJyIjpbXSwiYSI6WyJkaWRjb21tL3YyIl19
role:               Issuer
protocol_state:     OfferPending
publication_state:  <prism_agent_open_api_specification_client.types.Unset object at 0x7f3e60220580>
created_at:         2023-01-29 01:17:49.318241+00:00
updated_at:         <prism_agent_open_api_specification_client.types.Unset object at 0x7f3e60220580>


### Holder - Wait for credential offer

The holder waits to receive the credential offer. When received, it will show up in the holder's credential records list as a new entry with `protocol_state` equal to `OfferReceived`. The code below waits until a credential offer is received and takes the corresponding credential_record.

In [12]:
holder_credential_record = find_credential_record_by_state(holder_client, IssueCredentialRecordAllOfProtocolState.OFFERRECEIVED)

while(holder_credential_record == None):
    holder_credential_record = find_credential_record_by_state(holder_client, IssueCredentialRecordAllOfProtocolState.OFFERRECEIVED)
    time.sleep(1)
    
print_credential_record(holder_credential_record)

record_id:          0c263de4-99c6-4134-bc77-323fa868d386
subject_id:         did:peer:2.Ez6LSpzdmF4wrt3MEuMPxWTdL9TRkpTmKdgq3zqPsgEZ4giLi.Vz6MkgXLUEFc3LHcQ8kfA5c3DYhtXCkWBTSBuhF2m2Qinv8Jm.SeyJ0IjoiZG0iLCJzIjoiaHR0cHM6Ly9sdzIycS5hdGFsYXByaXNtLmlvL3ByaXNtLWFnZW50L2RpZGNvbW0iLCJyIjpbXSwiYSI6WyJkaWRjb21tL3YyIl19
role:               Holder
protocol_state:     OfferReceived
publication_state:  <prism_agent_open_api_specification_client.types.Unset object at 0x7f3e60220580>
created_at:         2023-01-29 01:17:52+00:00
updated_at:         <prism_agent_open_api_specification_client.types.Unset object at 0x7f3e60220580>


### Holder - Accept credential offer

The holder uses the `accept_credential_offer` endpoint to accept the credential offer. It must provide the `record_id` of the offer. Accepting the credential offer tells the Issuer that the credential can be issued.

In [13]:
holder_credential_record: Response[IssueCredentialRecord] = accept_credential_offer.sync(client=holder_client, record_id=holder_credential_record.record_id)
print_credential_record(holder_credential_record)

record_id:          0c263de4-99c6-4134-bc77-323fa868d386
subject_id:         did:peer:2.Ez6LSpzdmF4wrt3MEuMPxWTdL9TRkpTmKdgq3zqPsgEZ4giLi.Vz6MkgXLUEFc3LHcQ8kfA5c3DYhtXCkWBTSBuhF2m2Qinv8Jm.SeyJ0IjoiZG0iLCJzIjoiaHR0cHM6Ly9sdzIycS5hdGFsYXByaXNtLmlvL3ByaXNtLWFnZW50L2RpZGNvbW0iLCJyIjpbXSwiYSI6WyJkaWRjb21tL3YyIl19
role:               Holder
protocol_state:     RequestPending
publication_state:  <prism_agent_open_api_specification_client.types.Unset object at 0x7f3e60220580>
created_at:         2023-01-29 01:17:52+00:00
updated_at:         2023-01-29 01:24:56+00:00


### Issuer - Wait for credential request

The issuer waits to receive the credential request. When received, the credential record state will change to `RequestReceived` in the issuer's credential records list. The code below waits until a credential request is received and takes the corresponding credential_record.

In [14]:
issuer_credential_offer = get_credential_record.sync(client=issuer_client, record_id=issuer_credential_offer.record_id)

while(issuer_credential_offer.protocol_state != IssueCredentialRecordAllOfProtocolState.REQUESTRECEIVED):
    issuer_credential_offer = get_credential_record.sync(client=issuer_client, record_id=issuer_credential_offer.record_id)
    print("protocol_state: {}\n".format(issuer_credential_offer.protocol_state))
    time.sleep(1)
    
print_credential_record(issuer_credential_offer)

record_id:          6d02900b-d861-4d2e-a516-d4486424e0fb
subject_id:         did:peer:2.Ez6LSpzdmF4wrt3MEuMPxWTdL9TRkpTmKdgq3zqPsgEZ4giLi.Vz6MkgXLUEFc3LHcQ8kfA5c3DYhtXCkWBTSBuhF2m2Qinv8Jm.SeyJ0IjoiZG0iLCJzIjoiaHR0cHM6Ly9sdzIycS5hdGFsYXByaXNtLmlvL3ByaXNtLWFnZW50L2RpZGNvbW0iLCJyIjpbXSwiYSI6WyJkaWRjb21tL3YyIl19
role:               Issuer
protocol_state:     RequestReceived
publication_state:  <prism_agent_open_api_specification_client.types.Unset object at 0x7f3e60220580>
created_at:         2023-01-29 01:17:49+00:00
updated_at:         2023-01-29 01:25:00+00:00


### Issuer - Accept credential request (issue credential to holder)

The issuer uses the `issue_credential` endpoint to issue the credential. It must provide the `record_id`. When issued the record state changes to `CredentialSent` and the credential is send to the holder.

In [15]:
issuer_credential_offer: Response[IssueCredentialRecord] = issue_credential.sync(client=issuer_client, record_id=issuer_credential_offer.record_id)
print_credential_record(issuer_credential_offer)

record_id:          6d02900b-d861-4d2e-a516-d4486424e0fb
subject_id:         did:peer:2.Ez6LSpzdmF4wrt3MEuMPxWTdL9TRkpTmKdgq3zqPsgEZ4giLi.Vz6MkgXLUEFc3LHcQ8kfA5c3DYhtXCkWBTSBuhF2m2Qinv8Jm.SeyJ0IjoiZG0iLCJzIjoiaHR0cHM6Ly9sdzIycS5hdGFsYXByaXNtLmlvL3ByaXNtLWFnZW50L2RpZGNvbW0iLCJyIjpbXSwiYSI6WyJkaWRjb21tL3YyIl19
role:               Issuer
protocol_state:     CredentialPending
publication_state:  <prism_agent_open_api_specification_client.types.Unset object at 0x7f3e60220580>
created_at:         2023-01-29 01:17:49+00:00
updated_at:         2023-01-29 01:25:13+00:00


### Holder - Wait for credential

The holder waits to receive the credential. When received, it will be added to the holder's credential record and the `protocol_state` will be updated to `CredentialReceived`. The code below waits until a credential is received.

In [23]:
holder_credential_record = get_credential_record.sync(client=holder_client, record_id=holder_credential_record.record_id)

while(holder_credential_record.protocol_state != IssueCredentialRecordAllOfProtocolState.CREDENTIALRECEIVED):
    holder_credential_record = get_credential_record.sync(client=holder_client, record_id=holder_credential_record.record_id)
    print(f"protocol state: {holder_credential_record.protocol_state}")
    time.sleep(1)
    
print_credential_record(holder_credential_record)

record_id:          0c263de4-99c6-4134-bc77-323fa868d386
subject_id:         did:peer:2.Ez6LSpzdmF4wrt3MEuMPxWTdL9TRkpTmKdgq3zqPsgEZ4giLi.Vz6MkgXLUEFc3LHcQ8kfA5c3DYhtXCkWBTSBuhF2m2Qinv8Jm.SeyJ0IjoiZG0iLCJzIjoiaHR0cHM6Ly9sdzIycS5hdGFsYXByaXNtLmlvL3ByaXNtLWFnZW50L2RpZGNvbW0iLCJyIjpbXSwiYSI6WyJkaWRjb21tL3YyIl19
role:               Holder
protocol_state:     CredentialReceived
publication_state:  <prism_agent_open_api_specification_client.types.Unset object at 0x7f3e60220580>
created_at:         2023-01-29 01:17:52+00:00
updated_at:         2023-01-29 01:25:25+00:00


### JWT Credential

The JWT Credential is available in the holder credential record. The website https://jwt.io/ can be used to decode the credential.

In [31]:
print(base64.b64decode(holder_credential_record.jwt_credential).decode())

"eyJhbGciOiJFUzI1NksifQ.eyJpc3MiOiJkaWQ6cHJpc206ZGFiOTc0NmVlMGJkNTgyOTdhMTFmNTFmYmEwMjA2ZDYwM2YzNGNhZWQ3NzkzYjExZjk4ODZiM2Y4NDhiZjQ1ZDpDcjhCQ3J3QkVsd0tCMmx6YzNWcGJtY1FCRUpQQ2dselpXTndNalUyYXpFU0lBbEctS2U5bG5PaEZWaUdJcGRDNzNPcmNkcEk2MHp4MzRNS1JVcHNSQXJSR2lCc1kzWmI1eTBzSUpRV1F4SW5ZZFcxQmNIUzhzVmE2QU52YWU0aElBYVR4aEpjQ2dkdFlYTjBaWEl3RUFGQ1R3b0pjMlZqY0RJMU5tc3hFaUJuZGU1bUZiSFJVYlljTHQ1UXZCVkxBQjNsUEE0Yi1XRlpJcDc4SHJORlN4b2c5dzJ0S3ZYeWRLSlEwMXk2dVhfM08ySmc1YXNJLVYzNUJmeXE1d1ZkUms0Iiwic3ViIjoiZGlkOnBlZXI6Mi5FejZMU3B6ZG1GNHdydDNNRXVNUHhXVGRMOVRSa3BUbUtkZ3EzenFQc2dFWjRnaUxpLlZ6Nk1rZ1hMVUVGYzNMSGNROGtmQTVjM0RZaHRYQ2tXQlRTQnVoRjJtMlFpbnY4Sm0uU2V5SjBJam9pWkcwaUxDSnpJam9pYUhSMGNITTZMeTlzZHpJeWNTNWhkR0ZzWVhCeWFYTnRMbWx2TDNCeWFYTnRMV0ZuWlc1MEwyUnBaR052YlcwaUxDSnlJanBiWFN3aVlTSTZXeUprYVdSamIyMXRMM1l5SWwxOSIsIm5iZiI6MTY3NDk1NTUxNCwiZXhwIjoxNjc0OTU5MTE0LCJ2YyI6eyJjcmVkZW50aWFsU3ViamVjdCI6eyJmaXJzdG5hbWUiOiJKb2UiLCJiaXJ0aGRhdGUiOiIxMVwvMTFcLzE5OTkiLCJpZCI6ImRpZDpwZWVyOjIuRXo2TFNwemRtRjR3cnQzTUV1TVB4V1Rk